First, we sets up the environment by importing libraries and ensuring a folder exists to store all generated plots.

In [11]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, log_loss
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm
import os
import warnings
import joblib
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
PLOT_DIR = "Plots"
os.makedirs(PLOT_DIR, exist_ok=True)

Then, we split the dataset into train / validation / test subsets with stratification and normalizes all features using **StandardScaler**.

In [12]:
def load_data(file_path_x, file_path_y, test_size=0.2, val_size=0.2):
    X = np.load(file_path_x)
    y = np.load(file_path_y)

    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=(test_size + val_size), random_state=42, stratify=y
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp,
        test_size=test_size / (test_size + val_size),
        random_state=42,
        stratify=y_temp
    )

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    return X_train, X_val, X_test, y_train, y_val, y_test

For the theoretical analysis part, we increase the polynomial degree of the input features to test whether giving logistic regression enough nonlinear feature combinations allows it to capture the data’s underlying patterns and achieve accurate classification.

In [3]:
possible_n_vals = [10]
possible_e_vals = [9]

def run_poly_logistic_regression(n, e):

    X = np.load(f'Datasets/kryptonite-{n}-X.npy')
    y = np.load(f'Datasets/kryptonite-{n}-y.npy')
    
    # Shuffle and split the data
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    print(f"\n===== Running: n={n}, degree={e} =====")
    print(f"Train shape: {X_train.shape}")

    # Create polynomial features
    poly = PolynomialFeatures(degree=e)
    X_train_poly = poly.fit_transform(X_train)
    X_val_poly   = poly.transform(X_val)
    X_test_poly  = poly.transform(X_test)

    features = X_train_poly.shape[-1]
    print(f"Expanded features: {features}")

    # Initialize and fit logistic regression
    logreg = LogisticRegression(max_iter=50000, solver='lbfgs', C=0.5)
    print("Training logistic regression model...")
    logreg.fit(X_train_poly, y_train)
    print("✅ Model fitted")

    # Evaluate on the training data
    y_train_pred = logreg.predict(X_train_poly)
    y_train_proba = logreg.predict_proba(X_train_poly)
    train_acc = accuracy_score(y_train, y_train_pred)
    train_loss = log_loss(y_train, y_train_proba)

    # Evaluate on the validation set
    y_val_pred = logreg.predict(X_val_poly)
    y_val_proba = logreg.predict_proba(X_val_poly)
    val_acc = accuracy_score(y_val, y_val_pred)
    val_loss = log_loss(y_val, y_val_proba)

    # Evaluate on the test set
    y_test_pred = logreg.predict(X_test_poly)
    y_test_proba = logreg.predict_proba(X_test_poly)
    test_acc = accuracy_score(y_test, y_test_pred)
    test_loss = log_loss(y_test, y_test_proba)

    print(f"📊 Train acc={train_acc:.4f}, loss={train_loss:.4f} | Val acc={val_acc:.4f}, loss={val_loss:.4f} | Test acc={test_acc:.4f}, loss={test_loss:.4f}")

    return {
        "n": n,
        "degree": e,
        "features": features,
        "train_acc": train_acc,
        "train_loss": train_loss,
        "val_acc": val_acc,
        "val_loss": val_loss,
        "test_acc": test_acc,
        "test_loss": test_loss
    }

# Iterate through the values
results = []
for n in tqdm(possible_n_vals, desc="Datasets"):
    for e in tqdm(possible_e_vals, desc=f"Degrees for n={n}", leave=False):
        res = run_poly_logistic_regression(n, e)
        results.append(res)

# Print the results
for r in results:
    print(f"n={r['n']}, deg={r['degree']} → train_acc={r['train_acc']:.3f}, val_acc={r['val_acc']:.3f}, test_acc={r['test_acc']:.3f}")

Python(43201) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Datasets:   0%|          | 0/1 [00:00<?, ?it/s]


===== Running: n=10, degree=9 =====
Train shape: (12000, 10)
Expanded features: 92378
Training logistic regression model...
✅ Model fitted


Datasets: 100%|██████████| 1/1 [16:10<00:00, 970.04s/it]

📊 Train acc=0.9838, loss=0.1193 | Val acc=0.8878, loss=0.3786 | Test acc=0.8885, loss=0.3548
n=10, deg=9 → train_acc=0.984, val_acc=0.888, test_acc=0.888


We move on to our own MLP.

Here we define the following:
- BASE_PARAMS defines default settings for the MLP classifier.
- HPP_MAP overrides certain hyperparameters depending on the dataset dimensionality (n).

In [13]:
BASE_PARAMS = {
    "activation": "relu",
    "solver": "adam",
    "batch_size": 128,
    "max_iter": 1000,
    "early_stopping": True,
    "n_iter_no_change": 50,
    "random_state": 42,
    "verbose": False,
}

HPP_MAP = {
    20: {"hidden_layer_sizes": (256, 128, 64), "learning_rate_init": 0.001, "alpha": 0.1},
    18: {"hidden_layer_sizes": (128, 128, 64), "learning_rate_init": 0.002, "alpha": 0.1},
    16: {"hidden_layer_sizes": (128, 64), "learning_rate_init": 0.001, "alpha": 0.1},
    14: {"hidden_layer_sizes": (128, 64), "learning_rate_init": 0.002, "alpha": 0.1},
    12: {"hidden_layer_sizes": (128, 64), "learning_rate_init": 0.002, "alpha": 0.1},
    10: {"hidden_layer_sizes": (128, 64), "learning_rate_init": 0.002, "alpha": 0.1},
}

Function that saves the model.

In [14]:
def save_model(model, file_path):

    joblib.dump(model, file_path)
    print(f"Model saved to {file_path}")

For each dataset (kryptonite-n), we load te data and run 5-fold cross-validation. Then, we train the MLP model and store the results.

In [15]:
results = []
cv_scores_dict = {}

for n in range(10, 22, 2):
    if n not in HPP_MAP:
        continue

    print(f"\n===== Processing kryptonite-{n} =====")
    X_train, X_val, X_test, y_train, y_val, y_test = load_data(
        f"Datasets/kryptonite-{n}-X.npy",
        f"Datasets/kryptonite-{n}-y.npy",
        test_size=0.2, val_size=0.2
    )

    X_cv = np.vstack((X_train, X_val))
    y_cv = np.concatenate((y_train, y_val))

    hpp = BASE_PARAMS.copy()
    hpp.update(HPP_MAP[n])

    # Cross-validation
    cv_params = hpp.copy()
    cv_params["early_stopping"] = False

    model_cv = MLPClassifier(**cv_params)
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(model_cv, X_cv, y_cv, cv=kfold, scoring="accuracy", n_jobs=-1)
    cv_scores_dict[n] = cv_scores

    print(f"Cross-validation accuracies: {cv_scores}")
    print(f"Mean CV accuracy: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

    # Final training
    model = MLPClassifier(**hpp)
    model.fit(X_cv, y_cv)
    
    save_model(
    model,
    f"models/features-{n}-hidden-{hpp['hidden_layer_sizes']}-alpha-{hpp['alpha']}.joblib",
    )

    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)
    y_test_pred = model.predict(X_test)

    acc_train = accuracy_score(y_train, y_train_pred)
    acc_val = accuracy_score(y_val, y_val_pred)
    acc_test = accuracy_score(y_test, y_test_pred)

    results.append({
        "n": n,
        "cv_mean": cv_scores.mean(),
        "cv_std": cv_scores.std(),
        "train_acc": acc_train,
        "val_acc": acc_val,
        "test_acc": acc_test,
        "model": model,
    })

    print(f"Training Accuracy: {acc_train:.4f}")
    print(f"Test Accuracy (held-out): {acc_test:.4f}")


===== Processing kryptonite-10 =====
Cross-validation accuracies: [0.9553125 0.964375  0.95625   0.9609375 0.96125  ]
Mean CV accuracy: 0.9596 ± 0.0034
Model saved to models/features-10-hidden-(128, 64)-alpha-0.1.joblib
Training Accuracy: 0.9603
Test Accuracy (held-out): 0.9613

===== Processing kryptonite-12 =====
Cross-validation accuracies: [0.959375   0.95677083 0.95885417 0.95989583 0.95390625]
Mean CV accuracy: 0.9578 ± 0.0022
Model saved to models/features-12-hidden-(128, 64)-alpha-0.1.joblib
Training Accuracy: 0.9620
Test Accuracy (held-out): 0.9556

===== Processing kryptonite-14 =====
Cross-validation accuracies: [0.95245536 0.95491071 0.95580357 0.95580357 0.95357143]
Mean CV accuracy: 0.9545 ± 0.0013
Model saved to models/features-14-hidden-(128, 64)-alpha-0.1.joblib
Training Accuracy: 0.9669
Test Accuracy (held-out): 0.9650

===== Processing kryptonite-16 =====
Cross-validation accuracies: [0.93164062 0.9234375  0.9203125  0.93828125 0.93300781]
Mean CV accuracy: 0.9293 ±

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Cross-validation accuracies: [0.82203125 0.85609375 0.8446875  0.92609375 0.91796875]
Mean CV accuracy: 0.8734 ± 0.0413
Model saved to models/features-20-hidden-(256, 128, 64)-alpha-0.1.joblib
Training Accuracy: 0.9720
Test Accuracy (held-out): 0.9505


Finally, we generate the following plots:
- Accuracy vs dimensionality.
- Comparison of test vs target accuracy.
- learning curve (loss + validation accuracy across all datasets).
- confusion matrix for the test set (across all datasets).

In [16]:
plt.rcParams.update({
    "font.size": 13,
    "axes.labelsize": 14,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 12
})

dims = np.array([r["n"] for r in results])
cv_means = np.array([r["cv_mean"] for r in results])
cv_stds = np.array([r["cv_std"] for r in results])
test_acc = np.array([r["test_acc"] for r in results])
target_acc = np.array([0.94, 0.93, 0.92, 0.91, 0.80, 0.75])[:len(dims)]

# Plot 1: Accuracy vs Dimensionality
plt.figure(figsize=(8,5))
plt.plot(dims, cv_means, 'o-', color='steelblue', label='Mean CV Accuracy')
plt.fill_between(dims, cv_means - cv_stds, cv_means + cv_stds,
                 color='steelblue', alpha=0.15, label='CV ±1 SD')
plt.plot(dims, test_acc, 's--', color='darkorange', label='Test Accuracy')
plt.plot(dims, target_acc, 'r:', linewidth=2, label='Target Accuracy')
plt.xlabel("Input Dimensionality (n)")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(PLOT_DIR, "accuracy_vs_dim.png"), dpi=300)
plt.close()

# Plot 2: Test vs Target Accuracy
width = 0.35
plt.figure(figsize=(8,5))
plt.bar(dims - width/2, test_acc, width, label='Test Accuracy', color='cornflowerblue')
plt.bar(dims + width/2, target_acc, width, label='Target Accuracy', color='lightcoral')
plt.xlabel("Dataset Dimensionality (n)")
plt.ylabel("Accuracy")
plt.legend()
plt.ylim(0.7, 1.0)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(PLOT_DIR, "test_vs_target.png"), dpi=300)
plt.close()

# Per-Dataset Learning Curves and Confusion Matrices
for r in results:
    n = r["n"]
    model = r["model"]
    X_train, X_val, X_test, y_train, y_val, y_test = load_data(
        f"Datasets/kryptonite-{n}-X.npy",
        f"Datasets/kryptonite-{n}-y.npy",
        test_size=0.2, val_size=0.2
    )

    # Combined Training Loss + Validation Accuracy
    if hasattr(model, "loss_curve_"):
        plt.figure(figsize=(8, 5))

        # X-axis for training loss
        epochs = np.arange(1, len(model.loss_curve_) + 1)
        plt.plot(epochs, model.loss_curve_, color="royalblue", linewidth=2, label="Training Loss")

        # X-axis for validation accuracy
        if hasattr(model, "validation_scores_"):
            val_epochs = np.arange(1, len(model.validation_scores_) + 1)
            plt.plot(val_epochs, model.validation_scores_,
                     color="darkorange", linewidth=2, label="Validation Accuracy")
            
        plt.xlabel("Epoch", fontsize=11)
        plt.ylabel("Loss / Accuracy", fontsize=11)
        plt.legend(frameon=True, loc="best")
        plt.grid(True, linestyle="--", alpha=0.6)
        plt.tight_layout()
        plt.savefig(os.path.join(PLOT_DIR, f"learning_curve_kryptonite_{n}.png"), dpi=300)
        plt.close()

    # Confusion Matrix
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap='Blues', values_format="d")
    plt.tight_layout()
    plt.savefig(os.path.join(PLOT_DIR, f"cm_kryptonite_{n}.png"), dpi=300)
    plt.close()

print(f"\nAll training, evaluation, and plots saved in '{PLOT_DIR}/'")


All training, evaluation, and plots saved in 'Plots/'


Run the MLP on the hidden datasets.

In [ ]:
def predict(input_path: str, output_path: str, model):
    X = np.load(input_path)
    predictions = model.predict(X)
    np.save(output_path, predictions)
    print(f"Predictions saved to {output_path}")


os.makedirs("hiddenlabels", exist_ok=True)
for n in range(10, 22, 2):
    input_path = f"Datasets/hidden-kryptonite-{n}-X.npy"
    output_path = f"hiddenlabels/y_predicted_{n}.npy"
    predict(input_path, output_path, model)

Run KNN model for each dataset for comparioson.

In [9]:
KNN_PARAMS = {
    "n_neighbors": 6,
    "metric": "euclidean",
    "weights": "distance"
}

knn_results = []
knn_cv_scores_dict = {}

for knn_n in range(10, 22, 2):
    print(f"\n===== Processing KNN for kryptonite-{knn_n} =====")

    X_train_knn, X_val_knn, X_test_knn, y_train_knn, y_val_knn, y_test_knn = load_data(
        f"Datasets/kryptonite-{knn_n}-X.npy",
        f"Datasets/kryptonite-{knn_n}-y.npy",
        test_size=0.2, val_size=0.2
    )

    # Combine train + val for CV
    X_cv_knn = np.vstack((X_train_knn, X_val_knn))
    y_cv_knn = np.concatenate((y_train_knn, y_val_knn))

    # Standardize
    scaler_knn = StandardScaler()
    X_train_knn = scaler_knn.fit_transform(X_train_knn)
    X_val_knn = scaler_knn.transform(X_val_knn)
    X_test_knn = scaler_knn.transform(X_test_knn)
    X_cv_knn = scaler_knn.fit_transform(X_cv_knn)

    # Cross-validation
    model_cv_knn = KNeighborsClassifier(**KNN_PARAMS)
    kfold_knn = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores_knn = cross_val_score(model_cv_knn, X_cv_knn, y_cv_knn, cv=kfold_knn, scoring="accuracy", n_jobs=-1)
    knn_cv_scores_dict[knn_n] = cv_scores_knn

    print(f"Cross-validation accuracies: {np.round(cv_scores_knn, 4)}")
    print(f"Mean CV accuracy: {cv_scores_knn.mean():.4f} ± {cv_scores_knn.std():.4f}")

    # Final training on full train+val
    model_knn = KNeighborsClassifier(**KNN_PARAMS)
    model_knn.fit(X_cv_knn, y_cv_knn)

    # Evaluate
    y_train_pred_knn = model_knn.predict(X_train_knn)
    y_val_pred_knn = model_knn.predict(X_val_knn)
    y_test_pred_knn = model_knn.predict(X_test_knn)

    acc_train_knn = accuracy_score(y_train_knn, y_train_pred_knn)
    acc_val_knn = accuracy_score(y_val_knn, y_val_pred_knn)
    acc_test_knn = accuracy_score(y_test_knn, y_test_pred_knn)

    knn_results.append({
        "n": knn_n,
        "cv_mean": cv_scores_knn.mean(),
        "cv_std": cv_scores_knn.std(),
        "train_acc": acc_train_knn,
        "val_acc": acc_val_knn,
        "test_acc": acc_test_knn,
        "model": model_knn,
    })

    print(f"Training Accuracy: {acc_train_knn:.4f}")
    print(f"Test Accuracy (held-out): {acc_test_knn:.4f}")


===== Processing KNN for kryptonite-10 =====
Cross-validation accuracies: [0.9588 0.9609 0.9572 0.9612 0.9622]
Mean CV accuracy: 0.9601 ± 0.0018
Training Accuracy: 1.0000
Test Accuracy (held-out): 0.9613

===== Processing KNN for kryptonite-12 =====
Cross-validation accuracies: [0.8646 0.8659 0.8581 0.8568 0.8625]
Mean CV accuracy: 0.8616 ± 0.0036
Training Accuracy: 1.0000
Test Accuracy (held-out): 0.9092

===== Processing KNN for kryptonite-14 =====
Cross-validation accuracies: [0.6583 0.6308 0.6549 0.6484 0.6491]
Mean CV accuracy: 0.6483 ± 0.0095
Training Accuracy: 1.0000
Test Accuracy (held-out): 0.6986

===== Processing KNN for kryptonite-16 =====
Cross-validation accuracies: [0.448  0.4441 0.4373 0.452  0.4438]
Mean CV accuracy: 0.4450 ± 0.0049
Training Accuracy: 1.0000
Test Accuracy (held-out): 0.4495

===== Processing KNN for kryptonite-18 =====
Cross-validation accuracies: [0.4519 0.462  0.4557 0.4608 0.4498]
Mean CV accuracy: 0.4560 ± 0.0048
Training Accuracy: 1.0000
Test Acc